<a href="https://colab.research.google.com/github/Harsh23-glitch/Harsh23-glitch/blob/main/anxity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

import joblib


In [25]:
from google.colab import files
uploaded = files.upload()


Saving anxiety_clean.csv to anxiety_clean (1).csv
Saving depression_clean.csv to depression_clean (1).csv
Saving normal_clean.csv to normal_clean (1).csv


In [26]:
import pandas as pd

depression = pd.read_csv('depression_clean.csv')
anxiety = pd.read_csv('anxiety_clean.csv')
normal = pd.read_csv('normal_clean.csv')


In [27]:
import os
os.listdir()


['.config',
 'anxiety_clean.csv',
 'normal_clean.csv',
 'anxiety_clean (1).csv',
 'drive',
 'depression_clean (1).csv',
 'normal_clean (1).csv',
 'depression_clean.csv',
 'sample_data']

In [28]:
depression['label'] = 'depression'
anxiety['label'] = 'anxiety'
normal['label'] = 'normal'

df = pd.concat([depression, anxiety, normal], ignore_index=True)
df.head()


,tweet,label
0,i be just so tire of this my body be tire my m...,depression
1,you hide when you cry because you want to be s...,depression
2,i be just so tire of this my body be tire my m...,depression
3,she pretend like nothing happen because she wa...,depression
4,have you ever want to cry but no tear come out...,depression


In [29]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)          # remove links
    text = re.sub(r"[^a-zA-Z\s]", "", text)      # remove special chars
    text = re.sub(r"\s+", " ", text).strip()     # remove extra spaces
    return text


In [30]:
print(depression.columns)
print(anxiety.columns)
print(normal.columns)


Index(['tweet', 'label'], dtype='object')
Index(['tweet', 'label'], dtype='object')
Index(['tweet', 'label'], dtype='object')


In [31]:
df['tweet']


,tweet
0,i be just so tire of this my body be tire my m...
1,you hide when you cry because you want to be s...
2,i be just so tire of this my body be tire my m...
3,she pretend like nothing happen because she wa...
4,have you ever want to cry but no tear come out...
...,...
3077,grapes and barbecue popchips i think so
3078,cough sneeze be tho worst
3079,i can be your sad whore ahaha
3080,long piss be the best


In [32]:
depression['label'] = 'depression'
anxiety['label'] = 'anxiety'
normal['label'] = 'normal'

df = pd.concat([depression, anxiety, normal], ignore_index=True)
df.head()


,tweet,label
0,i be just so tire of this my body be tire my m...,depression
1,you hide when you cry because you want to be s...,depression
2,i be just so tire of this my body be tire my m...,depression
3,she pretend like nothing happen because she wa...,depression
4,have you ever want to cry but no tear come out...,depression


In [33]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['clean_text'] = df['tweet'].apply(clean_text)


In [34]:
X = df['clean_text']
y = df['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [36]:
from sklearn.cluster import KMeans

k = 3   # we want 3 mental states: normal, depression, anxiety

kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_train_vec)


KMeans(n_clusters=3, random_state=42)

In [37]:
train_clusters = kmeans.labels_


In [38]:
import numpy as np
unique, counts = np.unique(train_clusters, return_counts=True)
dict(zip(unique, counts))



{np.int32(0): np.int64(1692),
 np.int32(1): np.int64(721),
 np.int32(2): np.int64(52)}

In [39]:
df["cluster"] = kmeans.predict(vectorizer.transform(df["clean_text"]))

for i in range(3):
    print(f"\nCluster {i}:")
    print(df[df["cluster"] == i]["label"].value_counts())



Cluster 0:
label
normal        1890
anxiety        148
depression      96
Name: count, dtype: int64

Cluster 1:
label
normal        637
depression    140
anxiety       108
Name: count, dtype: int64

Cluster 2:
label
normal        49
anxiety       12
depression     2
Name: count, dtype: int64


In [40]:
cluster_to_label = {
    0: "normal",
    1: "depression",
    2: "normal"
}


In [43]:
import joblib


cluster_to_label = {
    0: "normal",
    1: "depression",
    2: "anxiety"
}


joblib.dump(kmeans, "kmeans_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")
joblib.dump(cluster_to_label, "cluster_map.pkl")

print("Saved: kmeans_model.pkl, vectorizer.pkl, cluster_map.pkl")
print(f"Cluster Mapping Used: {cluster_to_label}")

Saved: kmeans_model.pkl, vectorizer.pkl, cluster_map.pkl
Cluster Mapping Used: {0: 'normal', 1: 'depression', 2: 'anxiety'}


In [44]:
import gradio as gr
import joblib
import re


try:
    kmeans = joblib.load("kmeans_model.pkl")
    vectorizer = joblib.load("vectorizer.pkl")
    cluster_map = joblib.load("cluster_map.pkl")
except FileNotFoundError:
    print("Error: Model files not found. Ensure Step 11 was executed successfully.")
    raise


def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def predict_cluster(text):
    """Cleans text, transforms it, predicts the cluster, and maps it to a label."""
    if not text.strip():
        return "Please enter text to analyze."

    cleaned = clean_text(text)


    vec = vectorizer.transform([cleaned])


    cluster = kmeans.predict(vec)[0]


    prediction = cluster_map.get(cluster, "UNKNOWN_CLUSTER")

    return f"Predicted Mental State: {prediction.upper()} (Cluster: {cluster})"

ui = gr.Interface(
    fn=predict_cluster,
    inputs=gr.Textbox(lines=5, placeholder="Enter patient message or tweet..."),
    outputs="text",
    title="Mental Health Clustering System (K-Means)",
    description="Unsupervised K-Means clustering model to classify text into: NORMAL / DEPRESSION / ANXIETY"
)


ui.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d800fae245cd2d6bea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
